In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('jobs.csv')

df.head()

,title,money,knoladge,company,addition,city,link
0,Продавец-консультант остров часовых изделий ТЦ...,По договорённости,NaN,AllTime,['Опыт не нужен'],Челябинск,https://russia.superjob.ru/vacancy/search/?pag...
1,Охранник,351.0,NaN,ГК БРУ,"['Опыт не нужен', 'Сменный график']",Тамбов,https://russia.superjob.ru/vacancy/search/?pag...
2,Техник отдела испытаний,55.0,NaN,Московский завод ФИЗПРИБОР,['Полная занятость'],Подольск,https://russia.superjob.ru/vacancy/search/?pag...
3,Фармацевт / провизор (м. Отрадное),115.0,NaN,Ригла7.6,"['Опыт не нужен', 'Сменный график']","Москва, Каргопольская улица, 11к1",https://russia.superjob.ru/vacancy/search/?pag...
4,Монтер пути 3 разряда,130.25,NaN,"СПб ГУП ""Горэлектротранс""6.9","['Отклик без резюме', 'Опыт не нужен', 'Полная...",Санкт-Петербург,https://russia.superjob.ru/vacancy/search/?pag...


In [14]:
import weaviate
from weaviate.classes.config import Property, DataType
from sentence_transformers import SentenceTransformer
import pandas as pd

# assume df already exists with columns:
# title, money, knoladge, company, addition, city, link

# connect to your Weaviate Cloud instance
client = weaviate.connect_to_weaviate_cloud(
    cluster_url="irb4ztsbqxme6eyvsn5daa.c0.asia-southeast1.gcp.weaviate.cloud",
    auth_credentials=weaviate.auth.AuthApiKey("R2wxczNkVjUxQUg2ZlNNUl9FNnJCTmRFSkQ2UElRSStaNUttV1NMWlN2QUtsZ0JqTmNKQTRhTjdDemVFPV92MjAw")
)

# create collection if not already present
client.collections.create(
    name="Job",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="money", data_type=DataType.NUMBER),
        Property(name="knoladge", data_type=DataType.TEXT),
        Property(name="company", data_type=DataType.TEXT),
        Property(name="addition", data_type=DataType.TEXT),
        Property(name="city", data_type=DataType.TEXT),
        Property(name="link", data_type=DataType.TEXT)
    ]
)

# load a small local embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

collection = client.collections.get("Job")


In [15]:
df = df.fillna("")

# Define columns by type
text_cols = ["title", "knoladge", "company", "addition", "city", "link"]
num_cols = ["money"]

# Ensure all text columns are strings
for col in text_cols:
    df[col] = df[col].astype(str).fillna("")

# Clean up numeric columns
for col in num_cols:
    df[col] = (
        df[col]
        .replace("По договоренности", 0)
        .replace(r"[^\d.]+", "", regex=True)  # remove non-numeric characters
    )
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# Optional: ensure no infinities
df[num_cols] = df[num_cols].replace([float("inf"), float("-inf")], 0)

print(df.dtypes)
print(df.head())

# insert all rows from df
for _, row in df.iterrows():
    text = f"{row['title']} {row['knoladge']} {row['city']}"
    emb = model.encode(text).tolist()
    collection.data.insert(properties=row.to_dict(), vector=emb)

# perform semantic search
query = "python data analyst in Bishkek"
query_emb = model.encode(query).tolist()

results = collection.query.near_vector(query_emb, limit=5)

for r in results.objects:
    print(
        f"{r.properties['title']} | {r.properties['company']} | {r.properties['city']}"
    )

client.close()


title        object
money       float64
knoladge     object
company      object
addition     object
city         object
link         object
dtype: object
                                               title   money knoladge  \
0  Продавец-консультант остров часовых изделий ТЦ...    0.00            
1                                           Охранник  351.00            
2                            Техник отдела испытаний   55.00            
3                 Фармацевт / провизор (м. Отрадное)  115.00            
4                              Монтер пути 3 разряда  130.25            

                        company  \
0                       AllTime   
1                        ГК БРУ   
2    Московский завод ФИЗПРИБОР   
3                      Ригла7.6   
4  СПб ГУП "Горэлектротранс"6.9   

                                            addition  \
0                                  ['Опыт не нужен']   
1                ['Опыт не нужен', 'Сменный график']   
2                           

c:\Users\nnurs\AppData\Local\Programs\Python\Python313\Lib\site-packages\weaviate\warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
c:\Users\nnurs\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygments\regexopt.py:78: ResourceWarning: unclosed <ssl.SSLSocket fd=1944, family=2, type=1, proto=0, laddr=('192.168.0.222', 63120), raddr=('34.160.174.119', 443)>
  for group in groupby(strings, lambda s: s[0] == first[0])) \
c:\Users\nnurs\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygments\regexopt.py:78: ResourceWarning: unclosed <ssl.SSLSocket fd=1904, family=2, type=1, proto=0, laddr=('192.168.0.222', 61639), raddr=('34.160.174.119', 443)>
  for group in groupby(strings, lambda s: s[0] == first[0])) \
c:\Users\nnurs\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygments\regexopt.py:78:

ReadError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение